In [84]:
from langchain.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import os
from langchain.schema.runnable import RunnableMap

os.environ["OPENAI_API_KEY"] = "asojnwij"

chain = (
    PromptTemplate.from_template(
        """
As an AI, you excel at assessing whether you have sufficient information to answer a question. 
If the question is broad or generic, you should respond as ENOUGH INFO. 
However, if the question requires additional information for a comprehensive answer, your response should be "MORE INFO". 
Here's the user's query. Please evaluate whether you can provide an answer with the current information or if more details are needed.
Remember to only respond with "MORE INFO" if you need more information to answer the question.
Remember to only respond with "ENOUGH INFO" if you have enough information to answer the question.
<question>
{question}
</question>

Reponse:"""
    )
    | ChatOpenAI()
    | StrOutputParser()
)
def dummy(input):
    return "dummy"
sampleChain = PromptTemplate.from_template(
        """Simply echo what you are being asked in Question below. Start by saying MoreInfoChain. 
Question: {question}
Answer:"""
    ) | ChatOpenAI() | StrOutputParser()
MoreInfoChain = RunnableMap(
    {"documents": dummy, "question": RunnablePassthrough()}
    )  | {"answer" :  sampleChain}
LessInfo = (
    PromptTemplate.from_template(
        """Respond to the question in a fun, calm and prfessional fashion. 
        Make sure to write a comprehensive answer. 
        Start by saying LessInfo. 
Question: {question}
Answer:"""
    )
    | ChatOpenAI() | {"answer" : StrOutputParser()}
)

general_chain = (
    PromptTemplate.from_template(
        """
Respond to the question by having DEFAULT: as a start
<question>
{question}
</question>
"""
    ) | ChatOpenAI() | {"answer" : StrOutputParser()}
)

In [85]:
from langchain_core.runnables import RunnableBranch

branch = RunnableBranch(
    (lambda x: "more info" in x["topic"].lower(), MoreInfoChain),
    (lambda x: "enough info" in x["topic"].lower(), LessInfo),
    general_chain,
)

In [86]:
full_chain = {"topic": chain, "question": lambda x: x["question"]} | branch

for chunk in full_chain.stream({"question": "What are the most recent updates on ECCE tracking?"}):
    print(chunk.get("answer"))


More
Info
Chain
 The
 most
 recent
 updates
 on
 EC
CE
 tracking
.



In [89]:
chain.invoke({"question": "What are the most recent updates on ECCE tracking?"})

'MORE INFO'